In [1]:
import warnings
warnings.filterwarnings('ignore')

# Import Neccessary libraries
import numpy as np 
import pandas as pd 

# Import Model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Set the decimal format
pd.options.display.float_format = "{:.2f}".format

In [3]:
# Load the dataset
df = pd.read_csv("diabetes_prediction_dataset.csv")

In [4]:
# Handle duplicates
df = df.drop_duplicates()

In [5]:
# Remove Unnecessary value [0.00195%]
df = df[df['gender'] != 'Other']

In [6]:
# Recategorize smoking history
def recategorize_smoking(smoking_status):
    if smoking_status in ['never', 'No Info']:
        return 'non-smoker'
    elif smoking_status == 'current':
        return 'current'
    elif smoking_status in ['ever', 'former', 'not current']:
        return 'past_smoker'


In [7]:
df['smoking_history'] = df['smoking_history'].apply(recategorize_smoking)

In [8]:
# One-hot encoding
def perform_one_hot_encoding(df, column_name):
    dummies = pd.get_dummies(df[column_name], prefix=column_name)
    df = pd.concat([df.drop(column_name, axis=1), dummies], axis=1)
    return df


In [9]:
data = df.copy()
data = perform_one_hot_encoding(data, 'gender')
data = perform_one_hot_encoding(data, 'smoking_history')

In [10]:
# Split data into features and target variable
X = data.drop('diabetes', axis=1)
y = data['diabetes']

In [11]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
fixed_svm = SVC(C=1, gamma=0.1, kernel='rbf', decision_function_shape='ovo')
fixed_svm.fit(X_train, y_train)

SVC(C=1, decision_function_shape='ovo', gamma=0.1)

In [13]:
y_pred_fixed_svm = fixed_svm.predict(X_test)

In [14]:
print("Fixed SVM Model Accuracy: ", accuracy_score(y_test, y_pred_fixed_svm))
print(classification_report(y_test, y_pred_fixed_svm))

Fixed SVM Model Accuracy:  0.9554769582856548
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     17525
           1       0.95      0.52      0.68      1701

    accuracy                           0.96     19226
   macro avg       0.95      0.76      0.83     19226
weighted avg       0.96      0.96      0.95     19226

